In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./7.OEM_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:

    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./7.OEM_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1

                    try:
                        resp = requests.get(f'''https://productdesk-api.cellacore.net/legacy/cardone/Vehicle/Recommendation/?q=(productSKU.keyword:{input_.loc[a, 'Part Number']})&size=5000&useCache=true&auth={input_.loc[a, 'Auth']}''',
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_json = resp.json()

                # = = = = = = = = = = = = = = =

                list_make = [json_['fitment']['Vcdb']['MakeName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_model = [json_['fitment']['Vcdb']['ModelName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_region = [json_['fitment']['Vcdb']['RegionName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_submodel = [json_['fitment']['Vcdb']['SubModelName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_vehicle_type = [json_['fitment']['Vcdb']['VehicleTypeName'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_year = [int(json_['fitment']['Vcdb']['Year'].strip()) for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_list_aspiration, list_list_cylinder, list_list_displacement, list_list_fuel, list_list_liter = [], [], [], [], []
                list_list_engine = [json_['fitment']['Vcdb']['Engines'] for json_ in list_json]
                for list_engine in list_list_engine:
                    if list_engine:
                        list_aspiration, list_cylinder, list_displacement, list_fuel, list_liter = [], [], [], [], []
                        for engine in list_engine:
                            list_aspiration.append(engine['AspirationName'].strip() if 'AspirationName' in engine and engine['AspirationName'].strip() != '-' else '')
                
                            # = = = = = = = = = = = = = = =
                            
                            cylinder = ''
                            if 'BlockType' in engine and engine['BlockType'].strip() != '-':
                                cylinder += engine['BlockType'].strip()
                            if 'Cylinders' in engine and engine['Cylinders'] != 0:
                                cylinder += str(engine['Cylinders'])
                            list_cylinder.append(cylinder)
                            
                            # = = = = = = = = = = = = = = =
                
                            displacement = ''
                            if engine['CID'].strip() != '-':
                                displacement += f'''{engine['CID'].strip()}cid'''
                            elif engine['CC'].strip() != '-':
                                displacement += f'''/{engine['CC'].strip()}cc'''
                            list_displacement.append(displacement.removeprefix('/'))
                            
                            # = = = = = = = = = = = = = = =
                
                            list_fuel.append(engine['FuelTypeName'].strip() if 'FuelTypeName' in engine else '')
                
                            # = = = = = = = = = = = = = = =
                
                            list_liter.append(f'''{engine['Liter']}L''' if 'Liter' in engine and engine['Liter'] != 0 else '')
                    else:
                        list_aspiration, list_cylinder, list_displacement, list_fuel, list_liter = [''], [''], [''], [''], ['']
                
                    list_list_aspiration.append(list_aspiration)
                    list_list_cylinder.append(list_cylinder)
                    list_list_displacement.append(list_displacement)
                    list_list_fuel.append(list_fuel)
                    list_list_liter.append(list_liter)
    
                # = = = = = = = = = = = = = = =

                list_list_drive_type = []
                list_list_drive_type_2 = [json_['fitment']['Vcdb']['DriveTypes'] for json_ in list_json]
                for list_drive_type_2 in list_list_drive_type_2:
                    if list_drive_type_2:
                        list_drive_type = []
                        for drive_type_2 in list_drive_type_2:
                            list_drive_type.append(drive_type_2['DriveTypeName'].strip())
                    else:
                        list_drive_type = ['']
                
                    list_list_drive_type.append(list_drive_type)

                # = = = = = = = = = = = = = = =

                list_position = [json_['fitment']['acesCriteria']['PositionName'].strip() if 'PositionName' in json_['fitment']['acesCriteria'] else '' for json_ in list_json]

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame()
                for i in range(len(list_json)):
                    df_temp_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                                 'Part Number': input_.loc[a, 'Part Number'],
                                                 'Auth': input_.loc[a, 'Auth'],
                                                 'Region': list_region[i],
                                                 'Vehicle Type': list_vehicle_type[i],
                                                 'Fuel': list_list_fuel[i],
                                                 'Make': list_make[i],
                                                 'Model': list_model[i],
                                                 'Submodel': list_submodel[i],
                                                 'Cylinder': list_list_cylinder[i],
                                                 'Displacement': list_list_displacement[i],
                                                 'Liter': list_list_liter[i],
                                                 'Year': list_year[i],
                                                 'Aspiration': list_list_aspiration[i]})
                    for j in range(len(list_list_drive_type[i])):
                        df_temp_temp_temp = df_temp_temp.copy()
                        df_temp_temp_temp['Drive Type'] = list_list_drive_type[i][j]
                        df_temp_temp_temp['Position'] = list_position[i]
                        df_temp = pd.concat([df_temp, df_temp_temp_temp], ignore_index=True).fillna('')
                
                df_temp = df_temp.drop_duplicates(keep='first', ignore_index=True)
                df_temp = df_temp.sort_values(by=['Region', 'Vehicle Type', 'Fuel', 'Make', 'Model', 'Submodel', 'Cylinder', 'Displacement', 'Liter', 'Year', 'Aspiration', 'Drive Type', 'Position'],
                                              ascending=[True, True, True, True, True, True, True, True, True, True, True, True, True],
                                              ignore_index=True)
                
                df_temp['Row'] = [i+1 for i in range(len(df_temp))]
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Auth': input_.loc[a, 'Auth']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(10):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Row'],
                                                    ascending=[True, True],
                                                    ignore_index=True)
        output_correct[['No',
                        'Part Number',
                        'Auth',
                        'Row',
                        'Region',
                        'Vehicle Type',
                        'Fuel',
                        'Make',
                        'Model',
                        'Submodel',
                        'Cylinder',
                        'Displacement',
                        'Liter',
                        'Year',
                        'Aspiration',
                        'Drive Type',
                        'Position']].to_excel(f'''./8.vehicle/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./8.vehicle/0.error/{file.removesuffix('.xlsx')}-vehicle_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：error，尝试次数：72] - 100.00% - type_93.Transmission Control Module-vehicle_error > 16512.73-9037
[剩余数量：0] - [当前时间：09:43:54]

输出ing...

爬虫存在error

Done ~


In [2]:
list_file = sorted(list(os.walk('./8.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

[]

In [3]:
from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./8.vehicle/{file}',
              f'''./8.vehicle/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

Progress: 0it [00:00, ?it/s]

Done ~


In [4]:
list_file = sorted(list(os.walk('./8.vehicle'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

[]